# Overview



In [1]:
%%capture
!pip install transformers==4.38.2
!pip install accelerate==0.27.2
!pip install datasets==2.18.0
!pip install peft==0.9.0
!pip install bitsandbytes==0.42.0

In [9]:
import os
import torch
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

os.environ["MODEL_NAME"] = "google/gemma-2b-it"

torch.backends.cudnn.deterministic=True
# https://github.com/huggingface/transformers/issues/28731
torch.backends.cuda.enable_mem_efficient_sdp(False)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
!accelerate estimate-memory ${MODEL_NAME} --library_name transformers

Loading pretrained config for `google/gemma-2b-it` from `transformers`...
config.json: 100%|█████████████████████████████| 627/627 [00:00<00:00, 2.57MB/s]
┌────────────────────────────────────────────────────┐
│   Memory Usage for loading `google/gemma-2b-it`    │
├───────┬─────────────┬──────────┬───────────────────┤
│ dtype │Largest Layer│Total Size│Training using Adam│
├───────┼─────────────┼──────────┼───────────────────┤
│float32│   1.95 GB   │ 9.34 GB  │      37.38 GB     │
│float16│  1000.0 MB  │ 4.67 GB  │      18.69 GB     │
│  int8 │   500.0 MB  │ 2.34 GB  │      9.34 GB      │
│  int4 │   250.0 MB  │ 1.17 GB  │      4.67 GB      │
└───────┴─────────────┴──────────┴───────────────────┘


# Checking Data

In [4]:
import pandas as pd

csv_train=pd.read_csv('/kaggle/input/llm-prompt-recovery/train.csv')
csv_train.tail()

,id,original_text,rewrite_prompt,rewritten_text
0,-1,The competition dataset comprises text passage...,"Convert this into a sea shanty: """"""The competi...",Here is your shanty: (Verse 1) The text is rew...


In [5]:
csv_test=pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')
csv_test.tail()

,id,original_text,rewritten_text
0,-1,The competition dataset comprises text passage...,Here is your shanty: (Verse 1) The text is rew...


In [6]:
csv_sub=pd.read_csv('/kaggle/input/llm-prompt-recovery/sample_submission.csv')
csv_sub.tail()

,id,rewrite_prompt
0,9559194,Improve that text.


# Loading Model

In [7]:
# pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    llm_int8_enable_fp32_cpu_offload=True,
)

tokenizer = AutoTokenizer.from_pretrained(os.getenv("MODEL_NAME"))
model = AutoModelForCausalLM.from_pretrained(
    os.getenv("MODEL_NAME"),
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [10]:
input_text = "The Weather of Melbourne"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

2024-03-05 23:52:20.740867: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 23:52:20.740977: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 23:52:20.916408: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


<bos>Write me a poem about Machine Learning.

Machines, with eyes that scan the land,

